In [45]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [46]:
class LSTMe(nn.Module):
    
    def __init__(self,input_size, hidden_size, target_size):
        super(LSTMe,self).__init__()
        self.encode = nn.Embedding(77,200)
        self.lstm = nn.LSTM(200, hidden_size,)
        self.drop = nn.Dropout()
        self.fc = nn.Linear(hidden_size, target_size)
        self.hidden_size = hidden_size
        
    def forward(self,x,hidden):
        enco = self.encode(x)
        outp,h = self.lstm(enco.view(len(x),1,-1),hidden)
        outp = self.fc(outp.view(len(x),-1))
        outp = F.log_softmax(outp,1)
        return outp,h
        
    def init_w(self):
        h_0 = torch.zeros(1,1,self.hidden_size)
        c_0 = torch.zeros(1,1,self.hidden_size)
        return h_0,c_0

In [77]:
with open("./input.txt",encoding="utf-8") as f:
    data= f.read()
    
caracter = list(set(data))
data_len, caracter_len = len(data),len(caracter)
cr_to_ix = {cr:i for i,cr in enumerate(caracter)}
ix_to_cr = {i:cr for i,cr in enumerate(caracter)}
sequen_len = 200

p,n = 0,0

model = LSTMe(200,100,77)
# loss_function = nn.CrossEntropyLoss()
loss_function = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)
loss_pic=[]


while n<1001:
    
    # 记忆25个字母长度,
    if p+sequen_len >data_len or p==0 :
        p=0
    inputs = [cr_to_ix[c] for c in data[p:p+sequen_len]]
    targets = [cr_to_ix[c] for c in data[p+1: p+sequen_len+1]]
    
    hidden = model.init_w()
    
    model.train()

    # 训练参数
    inp = torch.tensor(inputs)
    tar = torch.tensor(targets,dtype=torch.long)

    model.zero_grad()
    optimizer.zero_grad()

    output,hidden = model(inp,hidden)

    loss = loss_function(output,tar)
    loss.backward()
    optimizer.step()
            
        
    if n%100==0:
        print("\n epoch{}/1000\n,loss:{:.4f}\n".format(n,loss))
        loss_pic.append(loss.item())
        
    if n%100 == 0:
        with torch.no_grad():                                
            model.eval()
            x = torch.randint(77,(1,))
            for i in range(200):
                h = model.init_w()
                output,h = model(x,h)
                _, pre = torch.max(output[-1:],1)
                x = torch.cat((x,pre))
#             txt = x
#             for i in range(60):
#                 h = model.init_w()
#                 x = x[1:]
#                 output,h = model(x,h)
#                 _,pre = torch.max(output[-1:],1)
#                 x = torch.cat((x,pre))
#                 txt = torch.cat((txt,pre))
                
            print("--- xup ---\n","".join([ix_to_cr[c] for c in x.tolist()]))
            
    n+=1
    p+=1
    
print("\n------------------\n------- end ------\n------------------\n")


 epoch0/1000
4.3505

--- xup ---
 O!KI))Feeeu+’f!o’Feeeeu+’f!o’Feeeeu+’f!o’Feeeeu+’f!o’Feeeeu+’f!o’Feeeeu+’f!o’Feeeeu+’f!o’Feeeeu+’f!o’Feeeeu+’f!o’Feeeeu+’f!o’Feeeeu+’f!o’Feeeeu+’f!o’Feeeeu+’f!o’Feeeeu+’f!o’Feeeeu+’f!o’Feeeeu+’f!o’Feee

 epoch100/1000
1.2312

--- xup ---
 zen my first rater ararinten my first rater arararining my first rater arararining my first rater arararining my first rater arararining my first rater arararining my first rater arararining my first r

 epoch200/1000
0.5494

--- xup ---
 Then medemed tarily-chosen hyperparameters) started to generate very nice looking descriptions of images that were on the edge of making sense. Sometimes the ratio of of imonge sene mate very nice look

 epoch300/1000
0.4316

--- xup ---
 ’ste of met of imaking sense that we on the edge of making sense. Sometimes the ratio of how simple your model is to the quality of the results you get out of it blows past of images that were on the e

 epoch400/1000
0.4114

--- xup ---
 ze mone d